In [1]:
%load_ext dotenv
%dotenv

# unset google vertex api
# import os
# for k in [
#     "GOOGLE_GENAI_USE_VERTEXAI",
#     "GOOGLE_CLOUD_PROJECT",
#     "GOOGLE_CLOUD_LOCATION",
# ]:
#     os.environ.pop(k, None)
# print("Vertex mode:", os.getenv("GOOGLE_GENAI_USE_VERTEXAI"))

In [2]:
# sample zinc dataset
from datasets.iterable_dataset import IterableDataset
from datasets import load_dataset

dataset: IterableDataset = load_dataset(
    "haydn-jones/ZINC20", split="test", streaming=True
)

samples = dataset.shuffle(seed=225).take(100)["smiles"]

Resolving data files:   0%|          | 0/787 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/96 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/93 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/787 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/96 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/93 [00:00<?, ?it/s]

In [3]:
from batchata import Batch

batch = Batch(
    results_dir="./data/",
    max_parallel_batches=5,
    items_per_batch=10,
).set_default_params(model="gemini-2.5-flash", temperature=0.0)

prompt_template = "Write the IUPAC name of this molecule:\n%s\nWrite the IUPAC name only. Do not write any comments."
for smiles in samples:
    batch.add_job(prompt=prompt_template % smiles)

In [4]:
# run the batch
run = batch.run(dry_run=True, print_status=True)

2026-01-27 20:02:14 - batchata.core.batch_run - INFO - Created temporary state file: /var/folders/5k/k24pr0ns75zcrcwfppb15tph0000gn/T/tmp40brqaj2.json
2026-01-27 20:02:14 - batchata.core.batch_run - INFO - === DRY RUN MODE ===
2026-01-27 20:02:14 - batchata.core.batch_run - INFO - This will show cost estimates without executing jobs
2026-01-27 20:02:14 - batchata.core.batch_run - INFO - Analyzing 100 pending jobs...
2026-01-27 20:02:14 - batchata.core.batch_run - INFO - 
Job breakdown:
2026-01-27 20:02:14 - batchata.core.batch_run - INFO - 
GeminiProvider (100 jobs):


ClientError: 401 UNAUTHENTICATED. {'error': {'code': 401, 'message': 'API keys are not supported by this API. Expected OAuth2 access token or other authentication credentials that assert a principal. See https://cloud.google.com/docs/authentication', 'status': 'UNAUTHENTICATED', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'CREDENTIALS_MISSING', 'domain': 'googleapis.com', 'metadata': {'method': 'google.ai.generativelanguage.v1beta.GenerativeService.CountTokens', 'service': 'generativelanguage.googleapis.com'}}]}}

In [ ]:
results = run.results()